# App: AI Patent Advisor
### Dataset: BigPatent (https://huggingface.co/datasets/NortheasternUniversity/big_patent)

### Features:
**Simplified Prior Art Search:** Instead of complex keyword searches, users can describe their invention in natural language. A RAG system retrieves relevant patents from BigPatent based on semantic similarity and provides them as context to an LLM. A prompt like, "Given this invention description [user input] and these similar patents [retrieved patents], summarize the most relevant prior art and potential novelty issues" can generate a concise report. This simplifies the search process and makes it accessible to non-patent experts.

**Competitive Technology Monitoring:** Users define a specific technology area or competitor. The system automatically retrieves newly published patents from BigPatent within that domain. A prompt such as, "Summarize the key innovations disclosed in these recently published patents [retrieved patents] related to [technology area/competitor]" provides a quick overview of competitive activity.

**Patent Claim Analysis & Comparison:** Input two or more patent claims. The system retrieves relevant contextual information from the full patent text within BigPatent. A prompt like, "Compare and contrast the scope of these patent claims [input claims] considering their full patent specifications [retrieved patent text]. Identify key differences and potential areas of overlap" facilitates detailed claim analysis.

**Automated Patent Summary Generation:** Input a full patent text. The system uses a prompt like, "Generate a concise, non-legal summary of this patent [input patent] highlighting the key innovation and potential applications." This quickly generates summaries suitable for business audiences or technical teams, saving time and resources.

**Patent Landscape Overview by CPC Class:** Users specify a CPC classification code. The system retrieves a sample of patents from BigPatent within that class. A prompt like, "Based on these patents [retrieved patents] within CPC class [input code], summarize the current state of the art, key players, and emerging trends" provides a quick overview of a specific technology domain. The structured nature of CPC codes simplifies retrieval and analysis.

In [2]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "False"

In [3]:
!pip -q install gradio 
!pip -q install datasets
!pip -q install transformers
!pip -q install langchain 
!pip -q install sentence_transformers 
!pip -q install langchain-community faiss-cpu
!pip -q install openai

In [4]:
!pip -q install bert_score
!pip -q install langchain_openai
!pip -q install tqdm
!pip -q install pandas
!pip -q install streamlit

In [5]:
import gradio as gr
import streamlit as st
from datasets import load_dataset
from transformers import pipeline
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
#from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.memory import ConversationBufferWindowMemory
from langchain.llms import HuggingFacePipeline
from sentence_transformers import SentenceTransformer
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from tqdm import tqdm
import pandas as pd
from langchain_core.documents import Document
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer

2024-12-06 15:19:55.709490: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-06 15:19:56.049626: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733498396.171055   39687 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733498396.213411   39687 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-06 15:19:56.556292: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [6]:
# ADD KEYS
os.environ["HUGGINGFACEHUB_API_TOKEN"] = 
os.environ["HF_TOKEN"] = 
os.environ["OPENAI_API_KEY"] = 

In [7]:
# Load a smaller subset of the 'g' CPC code patents for initial development (100 samples)
dataset = load_dataset("big_patent", "g", trust_remote_code=True, split="train[:300]")
texts = dataset["description"]

README.md:   0%|          | 0.00/9.71k [00:00<?, ?B/s]

big_patent.py:   0%|          | 0.00/5.50k [00:00<?, ?B/s]

In [8]:
os.system('rm -rf .cache/huggingface/hub/datasets--big_patent')

0

In [9]:
def get_open_LLM_response(model, tokenizer, input_prompt, max_length_input=1024, max_length_output=256):
    
    
    inputs = tokenizer(input_prompt, return_tensors='pt', padding="max_length",
        truncation=True,
        max_length=1024)
    print(len(inputs['input_ids'][0]))
    prediction = model.generate(**inputs, max_length=256)
    #with tokenizer.as_target_tokenizer():
    return tokenizer.batch_decode(prediction)[0]



In [10]:
text = """ CROSS REFERENCE TO RELATED APPLICATIONS 
       [0001]    This application claims benefit under 35 U.S.C. §119(a) of German Patent Application No. 10 2009 024 826.9-32, filed Jun. 13, 2009, the entire contents of which are incorporated herein by reference. 
       BACKGROUND OF THE INVENTION 
       [0002]    1. Field of the Invention 
         [0003]    The invention relates generally to a system for compensating electromagnetic interfering fields, and in particular to a system for magnetic field compensation having two sensors and a digital processor. 
         [0004]    2. Description of Related Art 
         [0005]    For compensating electromagnetic interfering fields, in particular magnetic interfering fields, feedback control systems are used in the very most cases, whereby one, or more sensors measure the amplitude of the interfering field for all three Cartesian space axes. The measuring signals of the sensors are fed to a control loop, which calculates control, or actuator signals from the measuring signals of the sensors, for devices generating magnetic fields. 
         [0006]    The magnetic field to be compensated may be the terrestrial magnetic field, or may be generated by other current-carrying devices being in the surrounding. 
         [0007]    Magnetic field compensation systems are for example used in connection with imaging systems using magnetic fields, for example in the case of scanning electron microscopes (SEM). 
         [0008]    In case of the mentioned devices for generating magnetic fields, it may be a matter of a current-carrying conductor, in the easiest case. Generally, one assumes interfering fields having far field characteristics, i.e. such fields, whose field amplitude does not essentially change within the range of 5 m. This assumption for example is true for interferences by rail vehicles. If the interfering fields are homogeneous in the range of interest, the compensation fields should be homogeneous, also. 
         [0009]    Pairs of so-called Helmholtz coils are preferably used for generating homogeneous compensation fields. At this, it is about two coils each being connected in the same direction, and having a distance to each other being equal to the half length of the edge (=coil diameter) (so-called Helmholtz condition). 
         [0010]    Furthermore, pairs of Helmholtz coils are used, whose distance to each other is equal to one length of the edge. If one pair of Helmholtz coils is used for each of the three space axes, the pairs of coils form a cube-shaped cage around the location, at which one, or more interfering fields shall be compensated. In case of such a coil arrangement, there indeed are field inhomogeneities in the interior of the cage, but these are acceptable in the most cases of application. 
         [0011]    A device for compensating magnetic fields is disclosed in U.S. Publication No 2005/019555A1 and has three coil pairs in a cage. The magnetic field to be compensated is measured and compensated, where an analog controller is used. 
         [0012]    Systems are also available, with which only one coil per space axis is used for generating the compensation field, however the compensation region, i.e. the region in which a good compensation is achieved, is considerably smaller than in the case of Helmholtz coils. 
         [0013]    Generally, one single magnetic field sensor is used for measuring the magnetic field at the place of interest. As an exception, there is a second sensor which is, however, used for diagnosis purposes. A single magnetic field sensor does not allow to detect, whether the magnetic field to be compensated is homogeneous, or inhomogeneous at the location of the object to be protected. 
         [0014]    It is a further problem when compensating electromagnetic interfering fields that it cannot be measured directly at the location at which the interfering field is to be compensated, since the object to be protected against interfering fields generally is at this location. 
         [0015]    A further problem arises, if two magnetic field compensation systems are arranged directly adjacent to one another. Then, undesired feedback effects may occur between the two systems. 
         [0016]    There are problems with the control systems in that these control systems can generally be optimized to single application. An adjustment to control tasks that are quite different, such as upon changes in the control configuration, is as a rule not possible or only in a restricted manner possible and/or is to be implemented with great difficulties. Furthermore non-linear control systems which may have a better interference field compensation than linear control systems, generally can only be implemented with high costs. When control circumstances change, the whole control circuit or the control loop would have to be newly calculated, designed and/or changed. In most cases, the direct user is not a position to do so. 
       SUMMARY OF THE INVENTION 
       [0017]    Therefore, it is an object of the invention to provide a system for compensating electromagnetic interfering fields with which system homogeneous as well as inhomogeneous magnetic fields may be compensated. 
         [0018]    It is a further object of the invention to perform a simulation of measuring electromagnetic interfering fields at the location of the object to be protected. 
         [0019]    It is a still further object of the invention to equalize potentially arising feedback effects in the case of using two magnetic field compensation systems in immediate vicinity. 
         [0020]    In detail, a system for compensating electromagnetic interfering fields is provided, which has two real triaxial magnetic field sensors, three pairs of compensation coils, and one control unit in order to protect an object against influences of an interfering field. It is preferred to design the control unit as a control processor such as a Digital Signal Processor DSP or a field programmable gate array FPGA. 
         [0021]    The six in total output signals of the two real sensors may be combined to three output signals of a virtual sensor, by means of a freely definable kind of averaging. By choosing the averaging algorithm properly, it can be achieved that the output signals of the virtual sensor represent the amplitude of the interfering field at the location of the object to be protected. 
         [0022]    The averaging takes place by means of the control system, which receives the six output signals of the two real magnetic field sensors via six inputs. 
         [0023]    For every sensor, the output signals of the two magnetic field sensors may be represented by a three-dimensional vector. These two vectors may be combined to six-dimensional vector, i.e. a 6×1 matrix. The averaging over the output signals of the two real sensors, i.e. calculating the output signals of the virtual sensor, may be described by a matrix multiplication: 
         [0000]    
       
      
       V=M·S  
      
       
         
           
             V: 6×1 matrix of the output signals of the virtual sensor; 
             M: 6×6 matrix describing the averaging over the output signals of the real sensors; and 
             S: 6×1 matrix of the output signals of the virtual sensor. 
           
         
       
     
         [0027]    The now available output signals (=virtual input signals of the control system) of the virtual sensor are used as an input for independent control loops operating in parallel. These control loops may be broadband, selective concerning a frequency range, or selective concerning a frequency, also. The control loops have control algorithms transforming the virtual input signals V into changed signals {circumflex over (V)}. At this, {circumflex over (V)} is a 6×1 matrix representing the in total six changed input signals of the control system. The control algorithm is described by an operator Ω. There are no limitations concerning the control algorithm being used. Accordingly, the operator Ω may not be a matrix so that nonlinear algorithms may also be used. Therefore, the transition to the modified signals {circumflex over (V)} is described by 
         [0000]        {circumflex over (V)} =Ω( V )
 
         [0028]    The matrix {circumflex over (V)} is multiplied by a 6×6 matrix L, in order to obtain control signals for the six coils, i.e. 
         [0000]    
       
      
       O=L·{circumflex over (V)} 
      
     
         [0000]    with:
 
L: 6×6 matrix for calculating the control signals O from the modified signals O=L·{circumflex over (V)}.
 """

#pred_text = tokenizer.batch_decode(prediction)
#pred_text

In [11]:
# model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-bigpatent")
# tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-bigpatent")
# input_prompt = "Summarize: " + text
# get_open_LLM_response(model, tokenizer, input_prompt, max_length_input=1024, max_length_output=256)

In [12]:
# dataset[0:50]['abstract']

In [13]:
#embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [14]:
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7fd09a0f1750>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7fd09a0f3d90>, model='text-embedding-3-large', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [15]:
os.system("rm -rf db")

0

In [16]:
def create_vectordb(texts, chunk_size=1024, chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, 
        chunk_overlap=chunk_overlap
    )
    
    docs = []
    id = 1
    for text in tqdm(texts, desc="Creating chunks"):
        splits = text_splitter.split_text(text)
        source = f"patent_{id}" 
        for chunk in splits:
            docs.append(Document(page_content=chunk, metadata={"source": source}))
        id += 1
    

    db = FAISS.from_documents(documents=docs,
                              embedding=embeddings)
    
    return db

#Create vectordb
#db = create_vectordb(texts) 
#print(f"FAISS VectorDB created with {len(db.index_to_docstore_id)} vectors.")

In [17]:
#save index
#db.save_local("faiss_index")

In [18]:
#Load vectordb from the saved index
vector_db = FAISS.load_local(
    "faiss_index", embeddings, allow_dangerous_deserialization=True
)

In [19]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

In [20]:
def get_qa_chain():
    
    retriever = vector_db.as_retriever(search_type="mmr",
      search_kwargs={"k": 4, "fetch_k":9}
    )
    qa_chain = RetrievalQA.from_llm(
        llm = ChatOpenAI(model_name="gpt-4o", temperature=0, max_tokens=1000),
        retriever=retriever,
        return_source_documents=True,
        verbose=True
    )
    # qa_chain = ConversationalRetrievalChain.from_llm(
    #     llm = ChatOpenAI(model_name="gpt-4o", temperature=0, max_tokens=1000),
    #     retriever=retriever,
    #     memory=memory,
    #     return_source_documents=True
    # )
    
    sys_prompt = "Answer the question based on the context provided only. Do not add any new information."
    qa_chain.combine_documents_chain.llm_chain.prompt.messages[0] = SystemMessagePromptTemplate.from_template(sys_prompt)
    print(f"qa chain: {qa_chain}")
    return qa_chain

In [21]:
# prompt: write python code to quantitatively compare human written summary with LLM generated summary using RougeScore and BERTScore

from rouge_score import rouge_scorer
from bert_score import score

def compare_summaries(human_summary, llm_summary):
    """
    Quantitatively compares a human-written summary with an LLM-generated summary
    using ROUGE and BERTScore.

    Args:
        human_summary (str): The human-written summary.
        llm_summary (str): The LLM-generated summary.

    Returns:
        dict: A dictionary containing ROUGE and BERTScore scores.
    """
    # ROUGE Score
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = scorer.score(human_summary, llm_summary)

    # BERTScore
    P, R, F1 = score([llm_summary], [human_summary], lang="en", verbose=True)
    bert_score = {
        "precision": P.mean().item(),
        "recall": R.mean().item(),
        "f1": F1.mean().item(),
    }
    return {
        "rouge": rouge_scores,
        "bert_score": bert_score,
    }


# Example usage
human_summary = "This is a human-written summary of a document."
llm_summary = "This is an LLM-generated summary of the same document."

comparison_results = compare_summaries(human_summary, llm_summary)
comparison_results

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.78 seconds, 1.28 sentences/sec


{'rouge': {'rouge1': Score(precision=0.5, recall=0.5555555555555556, fmeasure=0.5263157894736842),
  'rouge2': Score(precision=0.2222222222222222, recall=0.25, fmeasure=0.23529411764705882),
  'rougeL': Score(precision=0.5, recall=0.5555555555555556, fmeasure=0.5263157894736842)},
 'bert_score': {'precision': 0.9074332118034363,
  'recall': 0.9262628555297852,
  'f1': 0.9167513251304626}}

In [22]:
comparison_results['bert_score']['precision'], comparison_results['rouge']['rouge1'].fmeasure

(0.9074332118034363, 0.5263157894736842)

In [23]:
def generate_response(query, context):
    llm = ChatOpenAI(temperature=0.2, model_name="gpt-4o", max_tokens=256) # or gpt-4 if available

    prompt_template = """Answer the query by using the context provided only.\
    Do not add any additional information. If the context is not relevant enough then\
    say Context not relevant enough. query: {query}\n context: {context}
    """
    prompt = PromptTemplate(template=prompt_template, input_variables=["query", "context"])
    summary = llm(prompt.format(query=query, context=context))
    return summary.content

In [24]:
def generate_summary(text):
    llm = ChatOpenAI(temperature=0.2, model_name="gpt-4o", max_tokens=128) # or gpt-4 if available

    prompt_template = """Summarize: {text}
    """
    prompt = PromptTemplate(template=prompt_template, input_variables=["text"])

    summary = llm(prompt.format(text=text))
    return summary.content
    
def generate_and_compare_summaries(dataset, num_samples=3):
    """
    Generates summaries for a given number of text descriptions using ChatOpenAI's GPT-4o,
    compares them with the respective abstracts using BERTScore, and returns statistics.
    """

    results = []
    for i in tqdm(range(num_samples), desc="Generating and comparing summaries"):
        description = dataset["description"][i]
        abstract = dataset["abstract"][i]

      
        llm_summary = generate_summary(description)
        print(f"description: {description}")
        print(f"abstract: {abstract}")
        print(f"llm summary: {llm_summary}\n\n")
        # Compare summaries
        metrics = compare_summaries(abstract, llm_summary)
        results.append({
          "description_id": i,
          "bert_score_precision": metrics['bert_score']["precision"],
          "bert_score_recall": metrics['bert_score']["recall"],
          "bert_score_f1": metrics['bert_score']["f1"],
          "rouge1_precision": metrics['rouge']['rouge1'].precision,
          "rouge1_recall": metrics['rouge']['rouge1'].recall,
          "rouge1_f1": metrics['rouge']['rouge1'].fmeasure
        })
      
    return pd.DataFrame(results)


# Example usage:
#summary_results = generate_and_compare_summaries(dataset)

#print(summary_results.describe()) # Prints statistics for precision, recall, and F1 score

In [1]:
# summary_results = generate_and_compare_summaries(dataset)


In [26]:
def get_summarized_results(retrieval_query, generation_query, search_kwargs={"k": 3, "fetch_k":5}):
   
    retriever = vector_db.as_retriever(search_type="mmr", search_kwargs=search_kwargs)
    source_documents = retriever.invoke(retrieval_query)
    
    source_content_dict = {}
    summarized_results = ""
    source_metadata = ""
    for doc in source_documents:
        source = doc.metadata['source']
        if source not in source_content_dict:
            source_content_dict[source] = ""
        source_content_dict[source] += doc.page_content

    for source,context in source_content_dict.items():
        summary = generate_response(query=generation_query, context=context)
        summarized_results +=  f"Source: {source}\n" + summary + "\n\n"
        source_metadata += f"Source: {source}\nExcerpt: {context}\n\n"
        

    return (summarized_results, source_metadata)

In [27]:
def get_retrieved_docs_metadata(result):
    # Get metadata of retrieved documents
    retrieved_docs_metadata = ""
    if 'source_documents' in result:
        for doc in result['source_documents']:
            if hasattr(doc, 'metadata'):
                retrieved_docs_metadata += f"Metadata: {doc.metadata}\nExcerpt: {doc.page_content}\n\n"
    return retrieved_docs_metadata

In [28]:
def patent_summarization(patent_text, query):
    llm = ChatOpenAI(temperature=0.2, model_name="gpt-4o", max_tokens=256)

    prompt_template = """{query}
    
    Patent text: {text}"""
    
    prompt = PromptTemplate(template=prompt_template, input_variables=["query", "text"])
    
    response = llm(prompt.format(query=query, text=patent_text))
    return response.content


In [29]:
def prior_art_search(description):
    summarized_results, source_metadata = get_summarized_results(retrieval_query=description, 
                                                                 generation_query=description)
    return (summarized_results, source_metadata)

In [30]:
def competitive_monitoring(technology_area):
    summarized_results, source_metadata = get_summarized_results(retrieval_query=technology_area, 
                                                                 generation_query="Analyze the following text and provide: 1. Summary\n 2. Date of filing if available\n 3.Organization or company name if available.")
    return (summarized_results, source_metadata)

In [31]:
def claim_analysis(claim1, claim2):
    qa_chain = get_qa_chain()
    result = qa_chain(
        {
            "query": f"Compare and contrast these two claims:\nClaim 1: {claim1}\nClaim 2: {claim2}"
        }
    )
    retrieved_docs_metadata = get_retrieved_docs_metadata(result)
    return (result["result"], retrieved_docs_metadata)

In [32]:
def landscape_overview(cpc_code):
    summarized_results, source_metadata = get_summarized_results(retrieval_query=cpc_code, 
                                                                 generation_query="Summarize: ")
    return (summarized_results, source_metadata)

In [33]:
# Query Examples (replace with your actual examples if needed)
prior_art_suggestions = [
    "Methods for detecting diseases",
    "Techniques to perform image detection",
    "Method to perform age-estimation of a face",
    "Device for controlling flow of fluid in a microfluidic channel"
]

monitoring_suggestions = [  # G-specific examples
    "Emerging Trends in Location-Based Services for Field Technicians",
    "Competitive Landscape for Electromagnetic Interference Compensation",
    "Innovations in Specimen Tracking and Management in Healthcare"
]

claim_suggestions = [ # G-specific examples
    ["Claim 1: A method for measuring the speed of light using interferometry.", "Claim 2: A method for determining the refractive index of a material using interferometry."],
    ["Claim 1: A device for generating acoustic waves using piezoelectric materials.","Claim 2: A device for generating acoustic waves using magnetostrictive materials."],
    ["Claim 1: A system for controlling the temperature of a superconducting magnet.", "Claim 2: A system for controlling the magnetic field strength of a superconducting magnet."]
]


summary_suggestions = [ # G-specific examples
    ["REFERENCE TO RELATED APPLICATIONS  \n       [0001]     This application claims benefit of Application Ser. No. 60/481,847, filed Dec. 31, 2003, and of Application Ser. No. 60/561,754, filed Apr. 13, 2004.  \n         [0002]     The entire contents of the aforementioned applications are herein incorporated by reference. The entire contents of all United States Patents and published and copending Applications mentioned below are also herein incorporated by reference. \n     \n    \n     BACKGROUND OF THE INVENTION  \n       [0003]     This invention relates to electro-optic displays and to methods for driving such displays. More specifically, in one aspect this invention relates to electro-optic displays with simplified backplanes, and methods for driving such displays. In another aspect, this invention relates to electro-optic displays in which multiple types of electro-optic units are used to improve the colors available from the displays. The present invention is especially, though not exclusively, intended for use in electrophoretic displays.  \n         [0004]     Electro-optic displays comprise a layer of electro-optic material, a term which is used herein in its conventional meaning in the imaging art to refer to a material having first and second display states differing in at least one optical property, the material being changed from its first to its second display state by application of an electric field to the material. Although the optical property is typically color perceptible to the human eye, it may be another optical property, such as optical transmission, reflectance, luminescence or, in the case of displays intended for machine reading, pseudo-color in the sense of a change in reflectance of electromagnetic wavelengths outside the visible range.  \n         [0005]     In the displays of the present invention, the electro-optic medium will typically be a solid (such displays may hereinafter for convenience be referred to as \u201csolid electro-optic displays\u201d), in the sense that the electro-optic medium has solid external surfaces, although the medium may, and often does, have internal liquid- or gas-filled spaces. Thus, the term \u201csolid electro-optic displays\u201d includes encapsulated electrophoretic displays, encapsulated liquid crystal displays, and other types of displays discussed below."[:1024]+"...", "Analyze the following patent text and provide:\n1. Concise summary\n2. Key Invention\n3. Potential commercial applications"],
]

landscape_suggestions = [ # G-specific examples
    "CPC code: G01N (Measuring instruments; Measuring methods in general)",
    "CPC code: G21B (Nuclear reactors; accessories or details thereof)",
    "CPC code: G02B (Optical elements, systems, or apparatus)"
]

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# AI Patent Advisor Multi-Use-Case App\n ## Dataset: BigPatent")
    
    with gr.Tab("Patent Summarization"):
        
        summary_inputs = [gr.Textbox(lines=5, placeholder="Enter patent text"),
                          gr.Textbox(lines=2, placeholder="Enter query")]
        summary_examples = gr.Examples(summary_suggestions, summary_inputs) # Query suggestions
        summary_outputs = gr.Textbox(lines=3, label="Patent Summary")
        summary_iface = gr.Interface(
            fn=patent_summarization,
            inputs=summary_inputs,
            outputs=summary_outputs,
            flagging_mode='auto'
        )
    
    with gr.Tab("Prior Art Search"):
        prior_art_inputs = gr.Textbox(lines=2, placeholder="Enter your invention description...")
        
        prior_art_examples = gr.Examples(prior_art_suggestions, prior_art_inputs) # Query suggestions
        prior_art_outputs = [gr.Textbox(lines=5, label="Potential Prior Art"),
                     gr.Textbox(label="Document Metadata")
            ]
        prior_art_iface = gr.Interface(
            fn=prior_art_search,
            inputs=prior_art_inputs,
            outputs=prior_art_outputs,
            flagging_mode='auto'
        )

    with gr.Tab("Competitive Monitoring"):
        monitoring_inputs = gr.Textbox(placeholder="Enter technology area")
        monitoring_examples = gr.Examples(monitoring_suggestions, monitoring_inputs) # Query suggestions
        monitoring_outputs = [gr.Textbox(lines=5, label="Competitive Landscape"),
                     gr.Textbox(label="Document Metadata")
            ]
        monitoring_iface = gr.Interface(
            fn=competitive_monitoring,
            inputs=monitoring_inputs,
            outputs=monitoring_outputs,
            flagging_mode='auto'
        )

    with gr.Tab("Claims Comparison"):
        claim_inputs = [
                gr.Textbox(lines=2, placeholder="Enter claim 1"),
                gr.Textbox(lines=2, placeholder="Enter claim 2"),
            ]
        claim_examples = gr.Examples(claim_suggestions, claim_inputs) # Query suggestions
        claim_outputs = [gr.Textbox(lines=5, label="Claim Comparison"),
                     gr.Textbox(label="Document Metadata")
            ]
        claim_iface = gr.Interface(
            fn=claim_analysis,
            inputs=claim_inputs,
            outputs=claim_outputs,
            flagging_mode='auto'
        )  
        
    with gr.Tab("Landscape Overview"):
        landscape_inputs = gr.Textbox(placeholder="Enter CPC code")
        ladscape_examples = gr.Examples(landscape_suggestions, landscape_inputs) # Query suggestions
        landscape_outputs = [gr.Textbox(lines=5, label="Landscape Overview"),
                     gr.Textbox(label="Document Metadata")
            ]
        landscape_iface = gr.Interface(
            fn=landscape_overview,
            inputs=landscape_inputs,
            outputs=landscape_outputs,
            flagging_mode='auto'
        )
        
demo.launch(debug=True, share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://51e26eaea4c50e209d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
